TODO:
1. Найти распределение вероятностей для каждого состояния
2. Построить график количества людей в системе от времени
3. Построить график среднего времени нахождения клиента в системе от времени

In [161]:
import threading
import random
import time
from tqdm import tqdm

In [162]:
N_CLIENTS = 680

LAMBDA = 0.5
MU1 = 0.9
MU2 = 0.2

LOGGING = False

clients_list = []

event = threading.Event()
event.set()

In [163]:

timings = []

def get_state(chair1, chair2) -> str:
    if (not chair1.is_busy and not chair2.is_busy):
        return '00'
    if (not chair1.is_busy and chair2.is_busy):
        return '01'
    if (chair1.is_busy and not chair2.is_busy):
        return '10'
    if(chair1.is_blocked and chair2.is_busy):
        return 'b1'
    if (chair1.is_busy and chair2.is_busy):
        return '11'

In [164]:
class Client:
    def __init__(self, id):
        self.id = id
        self.start_time = None
        self.finish_time = None


In [165]:
class Chair1:
    def __init__(self):
        self.is_busy = False
        self.is_blocked = False
    
    def serving(self, client: Client):
        random.seed()
        ksi = random.expovariate(MU1)
        self.is_busy = True
        if LOGGING:
            print(f'client {client.id} is starting serving on the 1 chair')
        time.sleep(ksi)
        if LOGGING:
            print(f'client {client.id} finished serving on the 1 chair')
    

In [166]:
class Chair2:
    def __init__(self):
        self.is_busy = False
    
    def serving(self, client: Client):
        random.seed()
        ksi = random.expovariate(MU2)
        self.is_busy = True
        if LOGGING:
            print(f'client {client.id} is starting serving on the 2 chair')
        time.sleep(ksi)
        if LOGGING:
            print(f'client {client.id} finished serving on the 2 chair')

In [167]:
def append_timing(timings: list, state: str, time):
    if len(timings) != 0 and timings[-1][0] == state:
        timings.pop()
    timings.append((state, time))

In [168]:
def serving(client: Client, chair1: Chair1, chair2: Chair2, start_time):
    if LOGGING:
        print(f'CLIENT {client.id} STARTED SERVING')
    client.start_time = time.time()
    append_timing(timings, get_state(chair1, chair2), time.time() - start_time)
    chair1.serving(client)
    append_timing(timings, get_state(chair1, chair2), time.time() - start_time)
    if not event.is_set():
        chair1.is_blocked = True
        if LOGGING:
            print(f'client {client.id} is waiting for chair 2')
    event.wait()
    event.clear()
    append_timing(timings, get_state(chair1, chair2), time.time() - start_time)
    chair1.is_busy = False
    chair1.is_blocked = False
    # timings.append((get_state(chair1, chair2), time.time() - start_time))
    chair2.serving(client)
    append_timing(timings, get_state(chair1, chair2), time.time() - start_time)
    event.set()
    client.finish_time = time.time()
    clients_list.append(client)
    if LOGGING:
        print(f'CLIENT {client.id} FINISHED SERVING')

In [169]:
chair1 = Chair1()
chair2 = Chair2()

serving_start = time.time()

start_time = time.time()

for i in tqdm(range(N_CLIENTS)):
    random.seed()
    ksi = random.expovariate(LAMBDA)
    time.sleep(ksi)
    if not chair1.is_busy:
        x = threading.Thread(target=serving, args=(Client(i + 1), chair1, chair2, start_time))
        x.start()   
    else:
        if LOGGING:
            print(f'CLIENT {i + 1} WAS REJECTED')
    
x.join()

serving_end = time.time()

total_time = serving_end - serving_start

if LOGGING:
    print(f'Serving finished, total time: {total_time}')
    for client in clients_list: 
        print(f'id: {client.id}, time in system: {client.finish_time - client.start_time}')

 73%|███████▎  | 499/680 [18:09<04:42,  1.56s/it]

In [ ]:
# timings

In [ ]:
probs = {'00': .0,
         '01': .0,
         '10': .0,
         '11': .0,
         'b1': .0}

prev_time = 0
for item in timings:
    state = item[0]
    time = item[1]
    probs[state] += (time - prev_time) / total_time
    prev_time = time  
    
probs    

{'00': 0.002138761957750143,
 '01': 0.322005131502784,
 '10': 0.00013651616967442516,
 '11': 0.19724679404549111,
 'b1': 0.47847279632430056}